# <center >Accidents corporels de la route en 2019
### Nettoyage de la base de données

Les importations nécessaires :

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Permet de ne pas afficher certains messages Warning
import numpy as np
import matplotlib as plt
import seaborn as sns

Importation des quatre bases de données via l'adresse URL récupéré sur le site Data.gouv

In [2]:
df_carac = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/e22ba475-45a3-46ac-a0f7-9ca9ed1e283a", sep = ";")
df_lieux = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/2ad65965-36a1-4452-9c08-61a6c874e3e6", sep = ";")
df_usagers = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/36b1b7b3-84b4-4901-9163-59ae8a9e3028", sep = ";")
df_vehicules = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/780cd335-5048-4bd6-a841-105b44eb2667", sep = ";")

Nous conservons dans un premier temps, les variables utiles pour notre projet. Nous avons ainsi éliminé certaines variables telles que le numéro de la route, l'adresse postale, l'action du piéton lors de l'accident...

In [3]:
df_carac = df_carac[["Num_Acc", "jour", "mois", "an", "hrmn", "lum", "dep", "com", "agg", "int", "atm", "col", "lat", "long"]]
df_lieux = df_lieux[["Num_Acc", "catr", "circ", "nbv", "surf", "prof", "plan", "infra", "situ", "vma"]] 
df_usagers = df_usagers[["Num_Acc", "id_vehicule", "place", "catu", "sexe", "grav", "an_nais", "trajet", "secu1"]]
df_vehicules = df_vehicules[["Num_Acc", "id_vehicule", "catv", "obs", "obsm", "choc", "occutc"]]

A l'aide du descriptif des modalités, nous remplaçons les modalités par leur nom complet pour une meilleure compréhension.

In [4]:
# Variables de la base Caracteristiques
def modalite_caracteristiques(df):
    df["lum"]= df['lum'].map({1:'Plein jour',
                        2:'Crépuscule ou aube',
                        3: "Nuit sans éclairage public",
                        4: "Nuit avec éclairage public non allumé",
                        5: "Nuit avec éclairage public allumé"})
    df['agglo'] = df['agg'].map({1:'Hors agglomeration',
                             2:'Agglomeration'})
    del df['agg']
    
    df['int']=df['int'].map({1:'Hors intersection',
                        2:'Intersection en X',
                        3:'Intersection en T',
                        4:'Intersection en Y',
                        5:'Intersection à plus de 4 branches',
                        6:'Giratoire',
                        7:"Place",
                        8:'Passage à niveaux',
                        9:'Autre'})
    df['atm'] = df['atm'].map({1:'Normale',
                         2:'Pluie legere',
                         3:'Pluie forte',
                         4:'Neige',
                         5:'Brouillard',
                         6:'Vent fort',
                         7:'Eblouissant',
                         8:'Couvert',
                         9:'Autre'})
    df['collision'] = df['col'].map({1:'2 - frontale',
                                 2:'2-arriere',
                                 3:'2-cote',
                                 4:'3-chaine',
                                 5:'3-multiple',
                                 6:'autre',
                                 7:'Sans collision'})
    del df['col']

# Variables de la base Usagers
def modalite_usagers(df) :
    df['catu'] = df['catu'].map({1:'Conducteur',
                                 2:'Passager',
                                 3:'Pieton'})
    df['sexe'] = df['sexe'].map({1:'Homme',
                                 2:'Femme'})
    df['grav'] = df['grav'].map({1:'Indemne',
                                 2:'Tue',
                                 3:'Blesse hospitalise',
                                 4:'Blesse leger'})
    df['trajet'] = df['trajet'].map({1:"Domicile-Travail",
                                 2:'Domicile-Ecole',
                                 3:'Courses',
                                 4:'Utilisation Professionnelle',
                                 5:'Loisirs',
                                 8:'Autre'})

    # Variables de la base Lieux
def modalite_lieux(df) :
    df['catr'] = df['catr'].map({1:'Autoroute',
                                 2:'Route nationale',
                                 3:'Route départementale',
                                 4:'Voie Communale',
                                 5:'Hors réseau public',
                                 6:'Parc de stationnement',
                                 7:'Routes de métropole urbaine',
                                 9:'Autre'})
    df["circ"]=df["circ"].map({1:"A sens unique",
                              2:"Bidirectionnelle",
                              3:'A chaussées séparées',
                              4:"Avec voies d'affectation variable"})
    df["prof"]=df["prof"].map({1:"Plat",
                              2:"Pente",
                              3:'Sommet de côte',
                              4:"Bas de côte"})
    df["plan"]=df["plan"].map({1:"Rectiligne",
                              2:"En courbe à gauche",
                              3:'En courbe à droite',
                              4:"En S"})
    df['surf'] = df['surf'].map({1:'Normale',
                                 2:'Mouillée',
                                 3:'Flaques',
                                 4:'Inondée',
                                 5:'Enneigée',
                                 6:'Boue',
                                 7:'Verglecée',
                                 8:'Corps gras - huile',
                                 9:'Autre'})
    df['infra'] = df['infra'].map({0:'Aucun',
                                1:'Souterrain-tunnel',
                                 2:'Pont',
                                 3:'Bretelle',
                                 4:'Voie ferrée',
                                 5:'Carrefour aménagé',
                                 6:'Zone piétonne',
                                 7:'Zone de péage'})
    df['situ'] = df['situ'].map({1:'Sur chaussée',
                                 2:"Sur bande d'arrêt d'urgence",
                                 3:'Sur accotement',
                                 4:'Sur trottoir',
                                 5:'Sur piste cyclable',
                                 6:'Sur autre voie spéciale',
                                 8:'Autre'})
    

# Variables de la base Véhicules
def modalite_vehicules(df) :
    df['catv'] = df['catv'].map({0:'Autre',
                             1:'EDP',
                             2:'Deux roues',
                             3:'Voiture',
                             4:'Deux roues',
                             5:'Deux roues',
                             6:'Autre',
                             7:'Voiture',
                             8:'Voiture',
                             9:'Voiture',
                             10:'Voiture',
                             11:'Voiture',
                             12:'Voiture',
                             13:'Poids lourd',
                             14:'Poids lourd',
                             15:'Poids lourd',
                             16:'Poids lourd',
                             17:'Poids lourd',
                             18:'Transport en commun',
                             19:'Transport en commun',
                             20:'Autre',
                             21:'Autre',
                             30:'Deux roues',
                             31:'Deux roues',
                             32:'Deux roues',
                             33:'Deux roues',
                             34:'Deux roues',
                             35:'Autre',
                             36:'Autre',
                             37:'Transport en commun',
                             38:'Transport en commun',
                             39:'Transport en commun',
                             40:'Transport en commun',
                             41:'Deux roues',
                             42:'Deux roues',
                             43:'Deux roues',
                             50:'EDP',
                             60:'EDP',
                             80:'EDP',
                             99:'Autre'})
    df["choc"] = df["choc"].map({0: 'Aucun',
                                1:'Avant',
                                 2:"Avant droit",
                                 3:'Avant gauche',
                                 4:'Arrière',
                                 5:'Arrière droit',
                                 6:'Arrière gauche',
                                 7:'Côté droit',
                                 8:'Côté gauche',
                                 9:'Chocs multiples'})
                             
modalite_caracteristiques(df_carac)
modalite_usagers(df_usagers)
modalite_lieux(df_lieux)
modalite_vehicules(df_vehicules)

Au total, en 2019, il y eu 58 840 accidents corporels de la route qui ont concernés au total 100 710 véhicules et 132 977 personnes.

### Appropriation du jeu de données et structure de la base

In [5]:
#Fusion de la base de données :
#df = df_carac.merge(df_lieux, how='left', on=['Num_Acc'])
#df = df.merge(df_vehicules, how='inner', on=['Num_Acc'])
#df = df.merge(df_usagers, how='inner', on=['Num_Acc', 'id_vehicule'])

In [6]:
#df.columns

In [7]:
#df.head(5)

In [8]:
#df.dtypes

In [9]:
#print(df.describe(include='all'))

### Etude des valeurs manquantes :

Dans ce jeu de données, les valeurs non renseignées étaient préalablement indiquées par "-1". Elles sont maintenant notées "NaN" dans nos bases de données. \
La majorité des variables ne possèdent pas de valeurs manquantes. Quelques variables possèdent moins d'1% de valeurs manquantes (ex : collision, état de la surface de la route). Ensuite, les variables "Sens de circulation" (Circu) et "Infrastructure" possèdent 5% de valeurs manquantes. \
Enfin, nous ne pourrons pas utiliser la variable "Trajet" car cette dernière possède plus de 33% de valeurs manquantes.

In [28]:
# Pour plus de lisibilité, on créé des listes avec le nom des variables
var_carac = ["lum", "agglo", "int", "collision", "atm", 'jour', 'mois', 'hrmn', 'dep', 'com', 'lat', 'long']
var_lieux = ["catr", "circ", "surf", "prof", "plan", "infra", "situ", 'nbv', 'vma']
var_usagers = ["place", "catu", "sexe", "grav", "an_nais", "trajet"]
var_vehicules = ["catv", 'obs', 'obsm', 'choc', 'occutc']

#Fonction permettant de calculer l'effectif et le nombre de valeurs manquantes pour chaque variables.
def effectif_val_manquantes(df, colonne):
    for elem in colonne :
        # Effectif :
        eff = (df[elem].isna().sum()).round(2)
        # Fréquence : 
        freq = ((df[elem].isna().sum())/len(df[elem])*100).round(2)
        # Affichage : 
        print("Pour la variable " + elem + " :  " + freq.astype(str) + " % \n ce qui représente un effectif de :    " + eff.astype(str))
        print("\n")

In [10]:
    # A Executer pour avoir la fréquence de valeurs manquantes pour chaque variable       
#effectif_val_manquantes(df_carac, var_carac)
#effectif_val_manquantes(df_lieux, var_lieux)
#effectif_val_manquantes(df_usagers, var_usagers)
#effectif_val_manquantes(df_vehicules, var_vehicules)

### Etude des modalités rares :

En statistique, il n'est pas souhaitable d'avoir des modalités rares qui ont un effectif inférieur à 5%. Pour cela, nous avons regroupé les modalités entre elles en suivant deux méthodes différentes :

* <u> Première méthode :</u> Regrouper les modalités ayant un sens proche entre elles. Par exemple, "Intersection en X" et "Intersection en Y" correspondent à la modalité commune "Intersection".

* <u> Deuxième méthode :</u> regrouper les modalités représentant moins de 5% entre elles afin de créer une modalité "Autre".

Méthode 1 :

In [11]:
# Table CARACT
# La modalité "Avec éclairage non allumé" devient "Sans éclairage"
df_carac['lum']=df_carac['lum'].replace(["Nuit avec éclairage public non allumé"], "Nuit sans éclairage public")

# Variable intersection : on regroupe tous les types d'intersections
df_carac['int']=df_carac['int'].replace(["Intersection en X", "Intersection en Y", "Intersection en T", "Intersection à plus de 4 branches"], "Intersection")

# Variable collision :
df_carac["collision"] = df_carac["collision"].replace(["3-chaine", "3-multiple"], "3 vehicules")

    #Table USAGER :
df_usagers["trajet"] = df_usagers["trajet"].replace(["Domicile-Travail", "Domicile-Ecole", "Course"], "Domicile-Travail-Course")
    
    # Table LIEUX
# Variable circulation : 
df_lieux['circ']=df_lieux['circ'].replace(["Avec voies d'affectation variable"], "Bidirectionnelle")

# Prof :
df_lieux['prof'] = df_lieux['prof'].replace(["Bas de côte", "Sommet de côte"], "Pente")

# Plan : 
df_lieux['plan'] = df_lieux['plan'].replace(["En courbe à gauche", "En courbe à droite", "En S"], "Courbe")

    # Table VEHICULES
#Choc : 
df_vehicules['choc'] = df_vehicules['choc'].replace(["Arrière gauche", "Arrière droit"], "Arrière côté")

Méthode 2 :

In [12]:
# Pour plus de lisibilité, on créé des listes avec le nom des variables catégorielles
var_carac_cat = ["lum", "agglo", "int", "collision", "atm"]
var_lieux_cat = ["catr", "circ", "surf", "prof", "plan", "infra", "situ"]
var_usagers_cat = ["place", "catu", "sexe", "grav", "trajet"]
var_vehicules_cat = ["catv", 'choc']

#Fonction pour regrouper les modalités représentant moins de 5% de leur base de données ensemble :
def regroupement_modalites(df, colonne):
    for elem in colonne :
        # Calcul du pourcentage de chaque modalité
        frequence = df[elem].value_counts()/len(df[elem])*100
        for i in range(len(frequence)) :
            # Si la fréquence est inférieur à 5, on inpute la modalité "Autre"
            if frequence[i] < 5 :
                df.loc[df[elem]==frequence.index[i], elem] = "Autre"   

# Execution pour regrouper les modalités entre elles
regroupement_modalites(df_carac, var_carac_cat)
regroupement_modalites(df_lieux, var_lieux_cat)

En vérifiant les effectifs des différentes modalités, on remarque que la catégorie "Autre" de la variable __surf__ (Surface de la route) représente seulement 1,5%. Cette modalité sera regroupée avec la modalité "Mouillée" car elle correspond aux routes enneigées, verglacées, innondées, etc.

In [13]:
# Fonction pour calculer le pourcentage de chaque modalité :
def effectif_pourcentage(df, colonne):
    for elem in colonne :
        print("Pour la variable " + elem + " :")
        print((df[elem].value_counts())/len(df[elem])*100)
        print("\n")
        
# Executer la fonction pour nos 4 bases de données :
#effectif_pourcentage(df_carac, var_carac_cat)
#effectif_pourcentage(df_lieux, var_lieux_cat)
#effectif_pourcentage(df_usagers, var_usagers_cat)
#effectif_pourcentage(df_vehicules, var_vehicules_cat)

In [14]:
# Remplacer la modalité Autre par Mouillée pour la variable Surf
df_lieux['surf'] = df_lieux['surf'].replace(["Autre"], "Mouillée")

### Fusionner toutes les tables :
Nous fusionnons nos 4 bases de données afin d'avoir une base de données globale et complète. Il y a dans la table __df__ une ligne par usager. Ainsi, plusieurs observations peuvent correspondre à un même accident (dans le cas où l'accident a impliqué au moins 2 personnes).

In [15]:
# Fusion de la table caractéristique avec lieux et vehicules
df = df_carac.merge(df_lieux, how='left', on=['Num_Acc']).merge(df_vehicules, how='inner', on=['Num_Acc'])
# Fusion avec la table usagers
df = df.merge(df_usagers, how='inner', on=['Num_Acc', 'id_vehicule'])

## Ajout de variables 

### Les variables Age

On crée deux variables Age, l'une étant continue et l'autre catégorielle. Celles-ci correspondent à l'âge des personnes impliquées dans l'accident et sont créées à partir de la variable existante __an_nais__ (année de naissance).

In [16]:
# Création de la variable age en continu
df["age"]= 2019 - df["an_nais"]

In [17]:
# Séparation de la variable age en catégories
conditions_age = [
                    (0 <= df['age']) & (df['age'] <= 15),
                    (16 <= df['age']) & (df['age'] <= 30),
                    (31 <= df['age']) & (df['age'] <= 65),
                    (66 <= df['age'])]

# Nom des modalités
categories_age = ['enfant', 'jeune', 'adulte', 'personne agee']

# Création de la variable dans le dataframe
df['categorie_age'] = np.select(conditions_age, categories_age, default='NaN')

### Variable Obstacle

On crée une variable __obstacle__ qui correspond au fait qu'il y ait un obstacle fixe (ex : arbre, poteau, mur, etc) et/ou un obstacle mobile (ex : piéton, animal, etc) lors de l'accident.

In [31]:
# Régle d'attribution pour la variable Obstacle :
conditions_obstacle = [
                        (df['obs']>0) & (df['obsm']>0),
                        (df['obs']>0) & (df['obsm']<=0),
                        (df['obs']<=0) & (df['obsm']>0),
                        (df['obs']==0) & (df['obsm']==0)]

# Nom des modalités :
types_obstacle = ['1 obstacle mobile et 1 obstacle fixe', '1 obstacle fixe', '1 obstacle mobile', 'Pas dobstacle']

# Création de la variable dans le dataframe
df['obstacle'] = np.select(conditions_obstacle, types_obstacle, default="NaN")

### Variable Gravité

On ajoute une variable qui donne une note de gravité à chaque accident nommée __gravite__. 
- gravite = 0 si tout le monde ressort indemne de l'accident 
- gravite = 1 si il y a au moins un blessé léger 
- gravite = 2 si il y a au moins un blessé hospitalisé ou au moins un mort 

In [19]:
def calcul_gravite(l_grav):
    # prend en entrée la liste des grav d'un même accident l_grav
    note = 0
    if (('Tue' in l_grav) or ('Blesse hospitalise' in l_grav)):
        #on check si on a un 'Tue' ou un 'Blesse hospitalise' (var grav) => note=2
        note = 2
        return (note) 
    elif ('Blesse leger' in l_grav):
        #si non on check si on a un 'Blesse leger' => note = 1
        note = 1
        return (note)
    # si non => note = 0
    return(note)

#TEST OK 
#calcul_gravite(['Indemne','Blesse hospitalise','Blesse leger','Tue'])
#calcul_gravite(['Indemne','Blesse leger','Indemne'])
#calcul_gravite(['Indemne','Indemne','Indemne'])
#calcul_gravite([])

In [20]:
#on veut récupérer toutes les données de gravité pour chaque personne d'un meme accident (var Num_Acc)
#on crée une variable gravite remplie de zero dans df 
df = df.assign(gravite=0)

#on récupère tous les Num_Acc distincts
accidents = df['Num_Acc'].unique()

for num in accidents: 
    #on boucle sur les Num_Acc pour récupérer toutes les valeurs de grav pour un meme accident 
    l_grav = list(df[df['Num_Acc'] == num]['grav'])
    note = calcul_gravite(l_grav) #on applique pour chaque accident la fonction calcul_gravite 
    for i in df[df['Num_Acc'] == num].index:
        #on insert la valeur dans chaque case de la nouvelle variable 
        df['gravite'][i] = note

In [20]:
#print(df)

### Variable concernant le nombre de voitures et d'usagers impliqués dans l'accident

Comme nous souhaitons avec une base de données finale avec une ligne par accidents, nous ne pouvons pas garder les informations personnelles pour tous les usagers. Nous avons donc décidé de récupérer les informations présentes dans les tables __df_usagers__ et __df_vehicules__ en créant les variables ci dessous :
* Nombre de voitures impliquées dans l'accident
* Nombre d'usagers impliquées dans l'accident
* Indicatrice qui vaut 1 si un piéton est impliqué dans l'accident

In [21]:
#Calculer le nombre de voitures dans un accident :
df = df.assign(nb_voitures=0)
    # Créer une table qui associe l'id d'un accident avec le nombre de voitures impliquées
nb_vehicules = df.groupby(["Num_Acc", 'id_vehicule']).agg({"id_vehicule" : "count"})
nb_vehicules = nb_vehicules.assign(nb_vehicules=1)
nb_vehicules = nb_vehicules.groupby("Num_Acc").agg({"nb_vehicules" : "sum"})

    # Fusionner  cette table avec le df
df = df.merge(nb_vehicules, how = 'inner', on="Num_Acc")
del df['nb_voitures']

In [22]:
# Calculer le nombre d'usagers dans un accident :
df = df.assign(nb_usagers=0)
    # Associer le numéro de l'accident avec le nombre d'usagers dans une table
nb_usagers = df.groupby("Num_Acc").agg({"nb_usagers" : "count"}).reset_index()
del df["nb_usagers"]

    # Fusionner cette table avec le df
df = df.merge(nb_usagers, how = 'inner', on="Num_Acc")

In [26]:
# On créé une variable qui vaut 1 s'il y a un piéton
df["pieton"] = (df.catu == "Pieton").replace([False, True], [0, 1])

# Table qui associe Numéro de l'accident avec l'indicatrice piéton
nb_pietons = df.groupby("Num_Acc").agg({"pieton" : "max"}).reset_index()
del df["pieton"]

# On fusionne cette table avec la table globale
df = df.merge(nb_pietons, how = 'inner', on="Num_Acc")

## Création d'une base globale avec une ligne par accident

Afin de pouvoir réaliser un modèle, nous devons avoir une unique base de données et non quatre. Pour cela, nous avons décidé de garder une ligne par accident.

In [29]:
# On récupère les variables des tables Lieux et Caractériques et les variables que l'ont venont de créer
df_accidents = df[["Num_Acc"] + var_carac + var_lieux + ["pieton", "obstacle", "gravite", "nb_vehicules", "nb_usagers"]]

# On efface les doublons
df_accidents.drop_duplicates(keep = 'first', inplace=True)